In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np

from functionsFIMS import getAnalysisNumbers, plotPolyExamples
from runDataClass import runData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(runNos[0])
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)
    
    simData.plotCellGeometry()

In [ ]:
simData.plot2DFieldLines('Cathode')